# Get Background Cell Images
## Imports

In [1]:
import cv2
import numpy as np
import os

from config import *
from CellProcessor import read_image, process_image, get_bboxes, get_label_yolo

/Users/sapplab/Desktop/CellDataProcessing/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Make dir for images

In [2]:
IMAGES_BACKGROUND_DIR_PATH = IMAGES_PATH + "../"  + "Background_10p/"

try:
    os.mkdir(IMAGES_BACKGROUND_DIR_PATH)
except OSError:
    print("Dir already exists: " + IMAGES_BACKGROUND_DIR_PATH)

Dir already exists: ../new_data/Necroptosis/../Background_10p/


## For every image get iamges with no died cells

In [3]:
images = os.listdir(GREEN_PATH)
len_images = len(images)
curr_image_num = 1
num_images = 0

for image in images:
    print(str(curr_image_num) + "/" + str(len_images))
    curr_image_num+=1
    green_image_path = GREEN_PATH + image
    base_image_path = PHASE_PATH + image
    img, img_base = read_image(green_image_path, base_image_path)
    img_green = cv2.imread(green_image_path, cv2.IMREAD_UNCHANGED)

    img_base_h, img_base_w = img_base.shape[:2]

    CROP_WIDTH = 128
    CROP_HEIGHT = 128
    CROP_WIDTH_OVERLAP = 64
    CROP_HEIGHT_OVERLAP = 52
    CROP_WIDTH_SHIFT = CROP_WIDTH - CROP_WIDTH_OVERLAP
    CROP_HEIGHT_SHIFT = CROP_HEIGHT - CROP_HEIGHT_OVERLAP
    num_width_windows = (img_base_w - CROP_WIDTH)/CROP_WIDTH_SHIFT
    num_height_windows = (img_base_h - CROP_HEIGHT)/CROP_HEIGHT_SHIFT

    for width_window in range(int(num_width_windows)):
        for height_window in range(int(num_height_windows)):
            curr_topleft_x = width_window * CROP_WIDTH_SHIFT
            curr_topleft_y = height_window * CROP_HEIGHT_SHIFT
            img_crop = img[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]
            img_base_crop = img_base[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]
            img_green_crop = img_green[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]

            img_dilation = process_image(img_crop, 
                                contrast, 
                                brightness, 
                                threshold, 
                                i_erode, 
                                i_dialate, 
                                plot = False)          

            bboxes = get_bboxes(img_dilation)
            mean_img = (np.sum(img_dilation)/4177920) * 100.0
            if len(bboxes) == 0 and mean_img < 40.0 and num_images < 10000:
                num_images+=1          
                img_file = IMAGES_BACKGROUND_DIR_PATH + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".png"
                cv2.imwrite(img_file, img_base_crop)

1/322
2/322
3/322
4/322
5/322
6/322
7/322
8/322
9/322
10/322
11/322
12/322
13/322
14/322
15/322
16/322
17/322
18/322
19/322
20/322
21/322
22/322
23/322
24/322
25/322
26/322
27/322
28/322
29/322
30/322
31/322
32/322
33/322
34/322
35/322
36/322
37/322
38/322
39/322
40/322
41/322
42/322
43/322
44/322
45/322
46/322
47/322
48/322
49/322
50/322
51/322
52/322
53/322
54/322
55/322
56/322
57/322
58/322
59/322
60/322
61/322
62/322
63/322
64/322
65/322
66/322
67/322
68/322
69/322
70/322
71/322
72/322
73/322
74/322
75/322
76/322
77/322
78/322
79/322
80/322
81/322
82/322
83/322
84/322
85/322
86/322
87/322
88/322
89/322
90/322
91/322
92/322
93/322
94/322
95/322
96/322
97/322
98/322
99/322
100/322
101/322
102/322
103/322
104/322
105/322
106/322
107/322
108/322
109/322
110/322
111/322
112/322
113/322
114/322
115/322
116/322
117/322
118/322
119/322
120/322
121/322
122/322
123/322
124/322
125/322
126/322
127/322
128/322
129/322
130/322
131/322
132/322
133/322
134/322
135/322
136/322
137/322
138/322
139/